In [16]:
cd /mnt/beegfs/scratch/data/S82_standards/

/mnt/beegfs/scratch/data/S82_standards


In [17]:
ls

outputs/  S82_hipscat/  stripe82calibStars_v4.2.dat


In [27]:
!split -n 10 stripe82calibStars_v4.2.dat part_

In [3]:
%matplotlib inline

import hipscat
import hipscat_import
import pandas as pd
import multiprocessing as mp
import glob
from tqdm import tqdm
import os.path
import numpy as np
import healpy as hp
import glob

pd.set_option('display.max_columns', None)

In [4]:
files = glob.glob("part*")
print(len(files))

10


In [5]:
filesizes = list(tqdm(map(os.path.getsize, files), total=len(files)))

100%|█████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 6090.18it/s]


In [6]:
for f in filesizes: print(f/1024/1024)

23.451735496520996
23.451733589172363
23.451733589172363
23.451733589172363
23.451733589172363
23.451733589172363
23.451733589172363
23.451733589172363
23.451733589172363
23.451733589172363


In [10]:
thresh = 0.05 * 1024 # in megabytes
npfiles = np.array(files)
print(npfiles)
fs = np.array(filesizes) / 1024 / 1024 # in megabytes
x = (np.cumsum(fs) // thresh).astype(int)
u = np.unique(x)
print(u)
groups = dict()
for fileId in u:
    groups[f"outputs/trilegal-TESTDELETE-{fileId:06d}.parquet"] = npfiles[x == fileId]

['part_aj' 'part_ad' 'part_ag' 'part_ae' 'part_ab' 'part_af' 'part_ai'
 'part_ac' 'part_aa' 'part_ah']
[0 1 2 3 4]


In [11]:
ls outputs/

In [14]:
bandCols=['Nobs', 'mmed', 'mmu', 'msig', 'mrms', 'mchi2']
pd.read_table(files[1],header=None, comment='#', sep="\s+", names=['CALIBSTARS','RA', 'Dec', 'RArms', 'Decrms','Ntot','Ar']+
                                                                          ['u'+c for c in bandCols]+
                                                                          ['g'+c for c in bandCols]+
                                                                          ['r'+c for c in bandCols]+
                                                                          ['i'+c for c in bandCols]+
                                                                          ['z'+c for c in bandCols])

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_137574/952378224.py:2: SyntaxWarning: invalid escape sequence '\s'
  pd.read_table(files[1],header=None, comment='#', sep="\s+", names=['CALIBSTARS','RA', 'Dec', 'RArms', 'Decrms','Ntot','Ar']+


,CALIBSTARS,RA,Dec,RArms,Decrms,Ntot,Ar,uNobs,ummed,ummu,umsig,umrms,umchi2,gNobs,gmmed,gmmu,gmsig,gmrms,gmchi2,rNobs,rmmed,rmmu,rmsig,rmrms,rmchi2,iNobs,immed,immu,imsig,imrms,imchi2,zNobs,zmmed,zmmu,zmsig,zmrms,zmchi2
0,IBSTARS_0297419,320.523525,-0.055610,0.0275,0.0275,18,0.160,6,24.324,24.312,0.326,0.043,0.053,18.0,21.577,21.560,0.014,0.065,0.811,18.0,20.098,20.101,0.006,0.036,1.217,18.0,18.973,18.974,0.004,0.023,1.179,18.0,18.356,18.350,0.008,0.035,1.175
1,CALIBSTARS_0297420,320.527008,-0.116640,0.0374,0.0374,19,0.173,4,24.321,24.278,0.576,0.132,0.111,17.0,22.220,22.222,0.024,0.078,0.735,19.0,20.755,20.760,0.010,0.048,1.111,17.0,19.548,19.546,0.006,0.009,0.439,19.0,18.889,18.885,0.011,0.046,1.062
2,CALIBSTARS_0297421,320.529928,-0.011430,0.0357,0.0357,19,0.155,7,24.655,24.660,0.371,0.095,0.065,18.0,22.129,22.149,0.022,0.132,1.315,19.0,20.743,20.754,0.010,0.047,0.964,18.0,19.651,19.654,0.006,0.032,1.213,19.0,19.069,19.050,0.012,0.072,1.362
3,CALIBSTARS_0297422,320.532345,-0.200296,0.0457,0.0457,19,0.167,4,23.630,23.662,0.436,0.115,0.126,16.0,22.143,22.116,0.022,0.129,1.417,18.0,20.752,20.757,0.010,0.034,0.807,19.0,19.386,19.381,0.005,0.031,1.411,18.0,18.617,18.617,0.009,0.038,1.008
4,CALIBSTARS_0297423,320.538694,-0.122667,0.0298,0.0298,19,0.175,5,24.048,24.041,0.331,0.098,0.057,16.0,22.059,22.054,0.021,0.048,0.504,19.0,20.624,20.608,0.009,0.049,1.126,18.0,19.217,19.217,0.005,0.012,0.572,19.0,18.486,18.492,0.008,0.046,1.285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99143,CALIBSTARS_0396562,327.443945,1.077223,0.0289,0.0289,16,0.199,15,20.857,20.856,0.021,0.068,0.734,16.0,19.924,19.917,0.006,0.018,0.831,16.0,19.595,19.593,0.005,0.016,0.785,16.0,19.479,19.474,0.006,0.027,0.970,16.0,19.406,19.393,0.018,0.099,1.157
99144,CALIBSTARS_0396563,327.445210,1.120190,0.0274,0.0274,16,0.195,16,21.011,20.980,0.024,0.112,1.086,15.0,18.333,18.331,0.004,0.013,0.719,16.0,16.895,16.896,0.003,0.007,0.650,15.0,16.193,16.195,0.003,0.009,0.640,12.0,15.812,15.811,0.004,0.004,0.289
99145,CALIBSTARS_0396564,327.446551,1.245869,0.0527,0.0527,15,0.201,6,23.099,23.060,0.198,0.134,0.265,15.0,20.511,20.507,0.008,0.059,1.766,15.0,19.091,19.095,0.005,0.024,1.366,14.0,17.969,17.973,0.004,0.015,1.076,15.0,17.389,17.387,0.005,0.015,0.717
99146,CALIBSTARS_0396565,327.446898,1.088492,0.0331,0.0331,16,0.198,7,23.403,23.411,0.239,0.185,0.202,15.0,20.546,20.540,0.008,0.040,1.300,16.0,19.072,19.073,0.004,0.018,1.091,15.0,18.200,18.198,0.004,0.008,0.546,16.0,17.726,17.726,0.006,0.020,0.779


In [18]:
bandCols=['Nobs', 'mmed', 'mmu', 'msig', 'mrms', 'mchi2']
def compactify(out, inputs):
    # dfs = [ pd.read_csv(fn, usecols=columns, dtype=dict(schema)) for fn in inputs ]
    dfs = [ pd.read_table(fn,header=None, comment='#', sep="\s+", names=['CALIBSTARS','ra', 'dec', 'RArms', 'Decrms','Ntot','Ar']+
                                                                          ['u'+c for c in bandCols]+
                                                                          ['g'+c for c in bandCols]+
                                                                          ['r'+c for c in bandCols]+
                                                                          ['i'+c for c in bandCols]+
                                                                          ['z'+c for c in bandCols]) for fn in inputs ]
    df = pd.concat(dfs)
    df.to_parquet(out, index=False)

#for i, (out, inputs) in enumerate(groups.items()):
#    compactify(out, inputs)
#    if i > 5:
#        break

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_132844/2994693039.py:4: SyntaxWarning: invalid escape sequence '\s'
  dfs = [ pd.read_table(fn,header=None, comment='#', sep="\s+", names=['CALIBSTARS','ra', 'dec', 'RArms', 'Decrms','Ntot','Ar']+


In [22]:
def compactify_aux(args):
    return compactify(*args)

with mp.Pool(processes=96) as pool:
    for _ in tqdm(pool.imap_unordered(compactify_aux, groups.items()), total=len(groups)):
        pass

100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.95it/s]


In [23]:
ls outputs/ -lah

total 66M
drwxr-xr-x 2 lovro beegfs   5 Oct  9 14:17 ./
drwxr-xr-x 3 lovro beegfs  12 Oct  9 14:09 ../
-rw-r--r-- 1 lovro beegfs 14M Oct  9 14:18 trilegal-TESTDELETE-000000.parquet
-rw-r--r-- 1 lovro beegfs 14M Oct  9 14:18 trilegal-TESTDELETE-000001.parquet
-rw-r--r-- 1 lovro beegfs 14M Oct  9 14:18 trilegal-TESTDELETE-000002.parquet
-rw-r--r-- 1 lovro beegfs 14M Oct  9 14:18 trilegal-TESTDELETE-000003.parquet
-rw-r--r-- 1 lovro beegfs 14M Oct  9 14:18 trilegal-TESTDELETE-000004.parquet


In [33]:
import healpy as hp

NSIDE=2**5

def skymap_aux(fn):
    df = pd.read_parquet(fn, columns='ra dec'.split())
    ipix = hp.ang2pix(NSIDE, df["ra"], df["dec"], nest=True, lonlat=True)
    i, ct = np.unique(ipix, return_counts=True)
    return i, ct

img = np.zeros(hp.nside2npix(NSIDE), dtype=int)
fns = list(glob.glob('outputs/*TESTDELETE*.parquet'))
fsub = fns

with mp.Pool(processes=96) as pool:
    for i, ct in tqdm(pool.imap_unordered(skymap_aux, fsub), total=len(fsub)):
        img[i] += ct

hp.mollview(img+1, title="TRILEGAL sources per ring256", 
            coord='G', flip='geo', norm='log', nest=True)

  0%|                                                                             | 0/5 [00:00<?, ?it/s]


ValueError: THETA is out of range [0,pi]

In [27]:
import hipscat_import.pipeline as runner
from hipscat_import.catalog.arguments import ImportArguments
from hipscat_import.catalog.file_readers import ParquetReader

in_file_paths = glob.glob("outputs/trilegal-TESTDELETE*.parquet")
print(f"Input files: {len(in_file_paths)}")
#in_file_paths = glob.glob("outputs/trilegal-00000*.parquet")
in_file_paths.sort()
args = ImportArguments(
    output_artifact_name="trilegal",
    output_path="/home/lovro/scratch/data/TRILEGAL_hipscat_new_TESTDELETE",
    input_file_list=in_file_paths,
    file_reader=ParquetReader(),
    ra_column="ra",
    dec_column="dec",
    highest_healpix_order=10,
    # overwrite=True, # Gone in 0.3.1?
    catalog_type="object",
    sort_columns="ra",
    dask_n_workers=96,
    dask_tmp="/nvme/tmp/dask_tmp",
    tmp_dir="/nvme/tmp/tmp_dir",
    resume="/nvme/tmp/resume_tmp",
    pixel_threshold=5_000_000,
    add_hipscat_index=True    
)
runner.pipeline(args)

/nvme/lovro/miniconda3/envs/photoD-dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Input files: 5


Mapping   :  20%|███████████▍                                             | 1/5 [00:00<00:01,  3.43it/s]2024-10-09 14:20:54,348 - distributed.worker - WARNING - Compute Failed
Key:       map_to_pixels-63b3a11b2e5b9084febdea8c32e7ea86
Function:  map_to_pixels
args:      ()
kwargs:    {'input_file': 'outputs/trilegal-TESTDELETE-000003.parquet', 'resume_path': '/nvme/tmp/tmp_dir/trilegal/intermediate', 'file_reader': <hipscat_import.catalog.file_readers.ParquetReader object at 0x7f6cc5c82030>, 'mapping_key': 'map_3', 'highest_order': 10, 'ra_column': 'ra', 'dec_column': 'dec', 'use_hipscat_index': False}
Exception: "ValueError('THETA is out of range [0,pi]')"

Mapping   :  40%|██████████████████████▊                                  | 2/5 [00:04<00:07,  2.34s/it]2024-10-09 14:20:54,425 - distributed.worker - WARNING - Compute Failed
Key:       map_to_pixels-1bb3a536222dce0760522c2726101a1e
Function:  map_to_pixels
args:      ()
kwargs:    {'input_file': 'outputs/trilegal-TESTDELETE-000004

mapping task map_to_pixels-63b3a11b2e5b9084febdea8c32e7ea86 failed with message:
THETA is out of range [0,pi]
mapping task map_to_pixels-1bb3a536222dce0760522c2726101a1e failed with message:
THETA is out of range [0,pi]


RuntimeError: Some mapping stages failed. See logs for details.

Define which columns (and their dtypes) will be imported.

In [ ]:
columns=['ra', 'dec', 'gall', 'galb' , 'm_ini', 'logage', 'm_h', 'mu0', 'av', 'mratio', 'mass', 'logl', 'logte', 'logg', 
         'umag', 'gmag', 'rmag', 'imag', 'zmag', 'ymag', 'gc', 'label', 'ring256']
# columns=pd.read_csv(fn).columns
# df = pd.read_csv(fn, usecols=columns)
# schema = list(zip(df.dtypes.index, map(str, df.dtypes)))
types = []
trilegal_dtypes = np.loadtxt("dtypes.txt", delimiter=",", dtype=str)
for i in trilegal_dtypes[:, 1]:
    if i=='REAL':
        types.append(np.float32)
    elif i=='DOUBLE':
        types.append(np.float64)
    elif i=='SMALLINT':
        types.append(np.int16)
    elif i=='INTEGER':
        types.append(np.int32)
schema = list(zip(trilegal_dtypes[:,0].tolist(), types))

In [ ]:
filesizes = os.path.getsize()

In [ ]:
thresh = 6 * 1024 # in megabytes
npfiles = np.array(files)
fs = np.array(filesizes) / 1024 / 1024 # in megabytes
x = (np.cumsum(fs) // thresh).astype(int)
u = np.unique(x)
groups = dict()
for fileId in u:
    groups[f"outputs/trilegal-TESTDELETE-{fileId:06d}.parquet"] = npfiles[x == fileId]

In [ ]:
# allfiles = open("trilegal-datalab.list.txt").read().split()

# files = [ fn for fn in allfiles if fn.find('binary') == -1 ]

files = glob.glob("../data/TRILEGAL_DataLab/data/single/chunk_*/*")
print(len(files))